In [18]:
# Part 1
blocks = input_str.strip().split("\n\n")

state = {}
parents = {}
output_size = 0

for row in blocks[0].splitlines():
    (wire, s) = row.split(": ")
    state[wire] = int(s)

for row in blocks[1].splitlines():
    parts = row.split()
    parents[parts[4]] = (parts[0], parts[2], parts[1])
    if parts[4] not in state:
        state[parts[4]] = None
    if parts[4][0] == "z":
        output_size += 1


def resolve(wire):
    if state[wire] is not None:
        return state[wire]

    (a, b, op) = parents[wire]
    if op == "AND":
        return resolve(a) & resolve(b)
    if op == "OR":
        return resolve(a) | resolve(b)
    if op == "XOR":
        return resolve(a) ^ resolve(b)


result = 0
for z in range(output_size):
    key = f"z{z:02}"
    result = result | (resolve(key) << z)

print(result)

50411513338638


In [15]:
# Part 2
from math import log2, ceil
from itertools import combinations
blocks = input_str.strip().split("\n\n")

N_BITS = 45 
op_by_name = {}
name_by_op = {}

for row in blocks[0].splitlines():
    (wire, s) = row.split(": ")

for row in blocks[1].splitlines():
    parts = row.split()
    op_by_name[parts[4]] = (parts[0], parts[2], parts[1])

name_by_op = {value: key for key, value in op_by_name.items()}

# Given a bitwise op, find the wire name
def get_name(a, b, op):
    ab = (a, b, op)
    if ab in name_by_op:
        return ab

    ba = (b, a, op)
    if ba in name_by_op:
        return ba
    
    # No match, which means one of the inputs was swapped
    print("! Could not find match for", a, op, b)
    return False
    

last_carry_name = None
last_sum_name = None
swaps = []

def swap(a, b):
        s1 = op_by_name[a]
        s2 = op_by_name[b]
        op_by_name[a] = s2
        op_by_name[b] = s1
        name_by_op[s1] = b
        name_by_op[s2] = a

        swaps.extend([res_name, expected_output_name])
        print("swaps so far", sorted(swaps))


for i in range(0, 44):
    # print("Enter", i)

    # 1. Work out the sum and handle any carry from the previous bit

    ## sum = x[i] ^ y[i]
    expected_sum = get_name(f"x{i:02}", f"y{i:02}", "XOR")
    sum_name = name_by_op[expected_sum]
    # print(" -> sum", sum_name)

    ## res = sum ^ carry
    # skipped for the least significant bit
    if i > 0:
        expected_output_name = f"z{i:02}"
        expected_res = get_name(sum_name, last_carry_name, "XOR")
        # print("looking for", sum_name, "XOR", last_carry_name)
        
        if expected_res:
            res_name = name_by_op[expected_res]

            # Hack: in general, every lookup should have error handling like this.
            # For my input at least, this case covered 3/4 of the swaps.
            if res_name != expected_output_name:
                print(f"ERROR - bad res name. Found {res_name}, expected {expected_output_name}.")
                swap(res_name, expected_output_name)
               
        else:
            # Find the name that should go here
            print("ERROR - mismatched term")
            # Hack - did this by hand, as it was the last crash
            # Code would need to find the name that should go in this place



    
    # 2. Work out the carry into the next bit    
    # carry = x[i] & y[i] | (last_carry & sum)
    #         [ part 1  ] | [part 2          ] 
    expected_carry1 = get_name(f"x{i:02}", f"y{i:02}", "AND")
    carry1_name = name_by_op[expected_carry1]
    # print(" -> carry 1", carry1_name)

    if i > 0:
        expected_carry2 = get_name(last_carry_name,sum_name,"AND")
        carry2_name = name_by_op[expected_carry2]
        # print(" -> carry 2", carry2_name)
        
        expected_carry = get_name(carry1_name, carry2_name, "OR")
        last_carry_name = name_by_op[expected_carry]
        # print(" -> next carry in is called", last_carry_name)
    else:
        # z01 has no carry in term
        last_carry_name = carry1_name
    







ERROR - bad res name. Found vwr, expected z06.
swaps so far ['vwr', 'z06']
ERROR - bad res name. Found tqm, expected z11.
swaps so far ['tqm', 'vwr', 'z06', 'z11']
ERROR - bad res name. Found kfs, expected z16.
swaps so far ['kfs', 'tqm', 'vwr', 'z06', 'z11', 'z16']
! Could not find match for gfv XOR thv
ERROR - mismatched term
! Could not find match for thv AND gfv


KeyError: False

In [ ]:
# Misc analysis / throw away code 

# z00 (x00 ^ y00)
#
# z01 ((x00 & y00) ^ (y01 ^ x01))
#      (kqn      ) ^ (knr      )
# z02 ((((x00 & y00) & (y01 ^ x01)) | (x01 & y01)) ^ (y02 ^ x02))
#      ((  fhg                    ) | (stk      )) ^ (sbv      )    
# z03 ((((((x00 & y00) & (y01 ^ x01)) | (x01 & y01)) & (y02 ^ x02)) | (y02 & x02)) ^ (y03 ^ x03))

def resolve(state, parents, wire):
    if state[wire] is not None:
        return state[wire]

    (a, b, op) = parents[wire]
    if op == "AND":
        return resolve(state, parents, a) & resolve(state, parents, b)
    if op == "OR":
        return resolve(state, parents, a) | resolve(state, parents, b)
    if op == "XOR":
        return resolve(state, parents, a) ^ resolve(state, parents, b)


def get_value(state, parents, prefix):
    result = 0
    keys = [k for k in sorted(state.keys()) if k[0] == prefix]
    for key in keys:
        n = int(key[1:])
        result = result | (resolve(state, parents, key) << n)

    return result

def get_values(parents):
    state = init_state.copy()
    x = get_value(state, parents, "x")
    y = get_value(state, parents, "y")
    z = get_value(state, parents, "z")
    return (x, y, z)


def find_bad_bits(parents):
    (x, y, z) = get_values(parents)

    fxy = x + y

    print(f"x =     {x:014} {x:046b}")
    print(f"y =     {y:014} {y:046b}")
    print(f"x + y = {(fxy):014} {(fxy):046b}")
    print(f"z =     {z:014} {z:046b}")

    delta = fxy ^ z
    print(f"d =     {delta:14} {delta:046b}")

    bad_bits = [i for i in range(N_BITS) if delta & 2 ** i]
    
    for b in bad_bits:
        print(b, fxy & (2 ** b) and 1 or 0, z & (2 ** b) and 1 or 0)

    return bad_bits



In [2]:
# Test input 1
input_str="""
x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02
"""

In [15]:
# Test input 2
input_str="""
x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj
"""

In [1]:
# Test input 3 (AND)
input_str="""
x00: 0
x01: 1
x02: 0
x03: 1
x04: 0
x05: 1
y00: 0
y01: 0
y02: 1
y03: 1
y04: 0
y05: 1

x00 AND y00 -> z05
x01 AND y01 -> z02
x02 AND y02 -> z01
x03 AND y03 -> z03
x04 AND y04 -> z04
x05 AND y05 -> z00
"""

In [1]:
# Input
input_str="""
x00: 1
x01: 1
x02: 1
x03: 1
x04: 0
x05: 1
x06: 0
x07: 1
x08: 0
x09: 1
x10: 1
x11: 1
x12: 1
x13: 1
x14: 0
x15: 0
x16: 0
x17: 1
x18: 0
x19: 0
x20: 0
x21: 0
x22: 0
x23: 0
x24: 0
x25: 0
x26: 0
x27: 1
x28: 1
x29: 0
x30: 0
x31: 1
x32: 0
x33: 1
x34: 0
x35: 0
x36: 1
x37: 0
x38: 1
x39: 0
x40: 0
x41: 0
x42: 1
x43: 0
x44: 1
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1
y05: 0
y06: 0
y07: 0
y08: 0
y09: 0
y10: 1
y11: 0
y12: 0
y13: 1
y14: 0
y15: 0
y16: 1
y17: 0
y18: 1
y19: 0
y20: 1
y21: 0
y22: 1
y23: 0
y24: 0
y25: 0
y26: 0
y27: 0
y28: 0
y29: 0
y30: 1
y31: 1
y32: 0
y33: 1
y34: 1
y35: 0
y36: 1
y37: 0
y38: 0
y39: 1
y40: 1
y41: 0
y42: 0
y43: 1
y44: 1

x08 AND y08 -> hdk
y44 AND x44 -> gwr
vqt XOR wbv -> z43
gwr OR mmj -> z45
fsf AND jkd -> hgh
scp AND gbp -> ctg
vfr OR wqn -> gjt
hsd OR ktq -> tqr
mkh OR dsd -> cbb
jgv AND hbm -> brv
y03 XOR x03 -> kfr
mdm XOR mhn -> z32
mdm AND mhn -> snk
jgr AND gvg -> hnd
rpv OR ksg -> hpt
x00 AND y00 -> kqn
dsp OR jpc -> frp
hdv OR tcc -> jtf
qbg OR qbd -> vgg
y12 AND x12 -> cqg
x33 AND y33 -> twr
fqm OR hcq -> fsf
x38 AND y38 -> mps
kqn XOR knr -> z01
sgj XOR vfk -> z28
x09 XOR y09 -> dqr
cbb AND rmq -> ccj
smh AND gvv -> rpv
y16 AND x16 -> bmp
qqp XOR bjg -> z10
ckb OR wjk -> pcr
x19 AND y19 -> fqj
x07 AND y07 -> wjk
fhg OR stk -> mkg
bjg AND qqp -> dsp
y01 XOR x01 -> knr
jkt XOR ghh -> z44
gsb OR cqg -> rwd
y35 AND x35 -> jgm
qkd AND hjm -> qvr
hpt AND vgv -> vjc
vqr AND cth -> hcq
x12 XOR y12 -> ngj
y13 XOR x13 -> ktt
y09 AND x09 -> wtn
snk OR fmp -> mbg
ccj OR fdw -> gvh
y14 AND x14 -> fdn
rwd XOR ktt -> z13
x03 AND y03 -> ttq
tft AND vgg -> prq
x04 AND y04 -> pvm
khq AND nbj -> ccb
gfv OR wpj -> jgk
x36 AND y36 -> hcm
y02 AND x02 -> mrq
y41 XOR x41 -> rdd
ftk XOR wgn -> z24
hhk OR fqj -> rpq
x28 AND y28 -> wgg
mkg XOR sbv -> z02
thv XOR hcm -> z36
qfm OR vdn -> wbv
fps OR qgw -> kqb
tpc OR rdk -> mdm
jth OR brv -> bng
bmp OR vjc -> z16
thr XOR rpq -> z20
sct AND mbg -> gnq
mkg AND sbv -> wwd
x38 XOR y38 -> tft
y36 XOR x36 -> gfv
x15 XOR y15 -> gvv
x32 AND y32 -> fmp
x19 XOR y19 -> ndd
scp XOR gbp -> vwr
y28 XOR x28 -> sgj
x18 AND y18 -> fdw
wtn OR bgg -> bjg
y34 XOR x34 -> ssg
fcw XOR jgk -> z37
jtw XOR ssg -> z34
y11 XOR x11 -> sqv
y05 AND x05 -> qjv
pcr XOR kqg -> z08
jgr XOR gvg -> z30
qvr OR hfp -> qts
y25 XOR x25 -> rch
gfq XOR kfr -> z03
x11 AND y11 -> mjt
hnd OR vfp -> tdr
jgk AND fcw -> qbd
x21 XOR y21 -> hjm
x10 AND y10 -> jpc
y35 XOR x35 -> qsv
wgn AND ftk -> vfr
y39 XOR x39 -> khq
x20 AND y20 -> npb
kfr AND gfq -> fdc
mbg XOR sct -> z33
x41 AND y41 -> bdj
y15 AND x15 -> ksg
y42 AND x42 -> qfm
y32 XOR x32 -> mhn
gws OR hgh -> vfk
x34 AND y34 -> hsd
x06 AND y06 -> z06
svd AND ngj -> gsb
rpq AND thr -> bbd
pvm OR kdq -> ctv
vjr XOR jrr -> z29
gvh AND ndd -> hhk
dpf OR hdk -> nqv
jgv XOR hbm -> z40
bdj OR bhq -> wkv
tft XOR vgg -> z38
khq XOR nbj -> z39
y43 AND x43 -> qsd
y43 XOR x43 -> vqt
bng XOR rdd -> z41
tkp AND wkv -> vdn
x31 AND y31 -> tpc
x17 AND y17 -> mkh
fdf OR mbh -> ftk
ssg AND jtw -> ktq
x05 XOR y05 -> mpk
hmg OR jgm -> thv
rch AND gjt -> wsq
frp XOR sqv -> tqm
kqb XOR prr -> z14
y16 XOR x16 -> vgv
qkd XOR hjm -> z21
x24 XOR y24 -> wgn
kkp AND jcn -> ckb
x31 XOR y31 -> gpp
sgj AND vfk -> wnn
dqw AND jtf -> mbh
x40 XOR y40 -> jgv
vgv XOR hpt -> kfs
y40 AND x40 -> jth
dqr AND nqv -> bgg
ngj XOR svd -> z12
y22 AND x22 -> tcc
rdd AND bng -> bhq
x26 AND y26 -> fqm
x22 XOR y22 -> wqd
jrr AND vjr -> jtq
y23 XOR x23 -> dqw
y18 XOR x18 -> rmq
y39 AND x39 -> mtq
gpp AND tdr -> rdk
vwr OR ctg -> kkp
x23 AND y23 -> fdf
kqn AND knr -> fhg
kkp XOR jcn -> z07
x13 AND y13 -> qgw
tqr XOR qsv -> z35
y04 XOR x04 -> ffw
njk XOR kfs -> z17
kqb AND prr -> fgj
dph OR qsd -> ghh
tgb OR wsq -> vqr
nqv XOR dqr -> z09
wqd XOR qts -> z22
x37 AND y37 -> qbg
prq OR mps -> nbj
gjt XOR rch -> z25
jtq OR wnc -> jgr
mjt OR tqm -> svd
y17 XOR x17 -> njk
mpk XOR ctv -> z05
x25 AND y25 -> tgb
cth XOR vqr -> z26
x14 XOR y14 -> prr
wwd OR mrq -> gfq
y37 XOR x37 -> fcw
mtq OR ccb -> hbm
y29 AND x29 -> wnc
npb OR bbd -> qkd
y06 XOR x06 -> gbp
gpp XOR tdr -> z31
bnb OR qjv -> scp
y24 AND x24 -> wqn
y07 XOR x07 -> jcn
y10 XOR x10 -> qqp
y02 XOR x02 -> sbv
njk AND kfs -> dsd
mwb AND ffw -> kdq
fdn OR fgj -> smh
kqg AND pcr -> dpf
sqv AND frp -> z11
x44 XOR y44 -> jkt
ghh AND jkt -> mmj
tkp XOR wkv -> z42
y30 AND x30 -> vfp
x30 XOR y30 -> gvg
wbv AND vqt -> dph
y27 AND x27 -> gws
y08 XOR x08 -> kqg
y29 XOR x29 -> jrr
jkd XOR fsf -> z27
x01 AND y01 -> stk
gnq OR twr -> jtw
x26 XOR y26 -> cth
wnn OR wgg -> vjr
y42 XOR x42 -> tkp
ffw XOR mwb -> z04
gvv XOR smh -> z15
rwd AND ktt -> fps
qsv AND tqr -> hmg
x33 XOR y33 -> sct
fdc OR ttq -> mwb
x00 XOR y00 -> z00
mpk AND ctv -> bnb
rmq XOR cbb -> z18
qts AND wqd -> hdv
ndd XOR gvh -> z19
y27 XOR x27 -> jkd
thv AND hcm -> wpj
y21 AND x21 -> hfp
jtf XOR dqw -> z23
x20 XOR y20 -> thr
"""